<a href="https://colab.research.google.com/github/dbstj1231/2023_AI_Academy_ASR/blob/main/7_nemo_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

In [ ]:
import nemo.collections.asr as nemo_asr

In [ ]:
model = nemo_asr.models.ASRModel.from_pretrained('stt_en_conformer_ctc_large_ls')

In [ ]:
# token 추출

In [ ]:
# 단어 추출

In [ ]:
# word와 subword의 관계를 정리한 lexicon